In [5]:
from dotenv import load_dotenv
import os 
import pymongo

from enum import Enum
from pydantic import BaseModel 
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate

from langchain_core.pydantic_v1 import BaseModel, Field

load_dotenv()

True

In [2]:
class DrinkTaste(str, Enum):
    SWEET = "Sweet"
    SOUR = "Sour"
    BITTER = "Bitter"
    SALTY = "Salty"
    UMAMI = "Umami"
    FRUITY = "Fruity"
    FLORAL = "Floral"
    SPICY = "Spicy"
    CREAMY = "Creamy"
    TART = "Tart"
    REFRESHING = "Refreshing"
    RICH = "Rich"
    LIGHT = "Light"
    EARTHY = "Earthy"
    CITRUSY = "Citrusy"
    HERBAL = "Herbal"
    
class TasteResponce(BaseModel): 
    tastes: list[DrinkTaste] = Field(description=f"Drink tastes like, choose options: {' '.join([x.value for x in DrinkTaste])}", required=True)
        
prompt = PromptTemplate.from_template(
    """Extract the suitable tastes from the provided drink recipe.
    Respond with true or false for each taste based on the recipe.
    
    Possible options:
    SWEET = "Sweet"
    SOUR = "Sour"
    BITTER = "Bitter"
    SALTY = "Salty"
    UMAMI = "Umami"
    FRUITY = "Fruity"
    FLORAL = "Floral"
    SPICY = "Spicy"
    CREAMY = "Creamy"
    TART = "Tart"
    REFRESHING = "Refreshing"
    RICH = "Rich"
    LIGHT = "Light"
    EARTHY = "Earthy"
    CITRUSY = "Citrusy"
    HERBAL = "Herbal"
    
    Recipe: {recipe}
    AI: """
)

model = ChatOllama(model='mistral').with_structured_output(TasteResponce)

In [3]:
client = pymongo.MongoClient(os.getenv('MONGO_CONNECTION_STRING'))
db = client["monin"]
collection = db["drinks"]

In [4]:
for drink in collection.find({})[:1]:
    recipe = drink['name'] + "\n" + " ".join(drink['recipie'])
    tastes = model.invoke(prompt.format(recipe=recipe))
    
    tastes = list(tastes)[0][1]
    tastes = [taste.value for taste in tastes]
    print(tastes)
    print()

['Sweet', 'Refreshing', 'Creamy']

